<a href="https://colab.research.google.com/github/r-dube/fakejobs/blob/main/fj_lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Load the modules used
import numpy as np
import pandas as pd
import re
import string
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional, GlobalMaxPool1D
from keras.preprocessing.sequence import pad_sequences
from keras.initializers import Constant 
from keras.optimizers import Adam
from keras import metrics
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from collections import Counter

In [ ]:
# NLTK to remove stopwords
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# For reproducible results
import random as rn
import os
import tensorflow as tf
os.environ['PYTHONHASHSEED'] = '42'
os.environ['CUDA_VISIBLE_DEVICES'] = ''
np.random.seed(42)
rn.seed(42)
tf.random.set_seed(42)

In [ ]:
# Set data_url, the location of the data
# Data is not loaded from a local file
# data_url="https://raw.githubusercontent.com/r-dube/fakejobs/main/data/fj_small.csv"
# data_url="https://raw.githubusercontent.com/r-dube/fakejobs/main/data/fj_medium.csv"
data_url="https://raw.githubusercontent.com/r-dube/fakejobs/main/data/fake_job_postings.csv"

In [ ]:
def fj_load_df_from_url():
    """
    Load dataframe from csv file
    Input:
        None
    Returns:
        dataframe
    """

    df = pd.read_csv(data_url)

    print ('Loaded dataframe shape', df.shape)

    counts = fj_label_stats(df)
    print ('Not fraudulent', counts[0], 'Fraudulent', counts[1])

    print(df.describe())

    print ('NAs/NANs in data =>')
    print(df.isna().sum())

    return df

def fj_label_stats(df):
    """
    Very basic label statistics
    Input: 
        Dataframe
    Returns:
        Number of samples with 0, 1 as the label
    """
    counts = np.bincount(df['fraudulent'])
    return counts

def fj_txt_only(df):
    """
    Combine all the text fields, discard everything else except for the label
    Input: 
        Dataframe
    Returns:
        Processed dataframe
    """
    
    df.fillna(" ", inplace = True)

    df['text'] = df['title'] + ' ' + df['location'] + ' ' + df['department'] + \
    ' ' + df['company_profile'] + ' ' + df['description'] + ' ' + \
    df['requirements'] + ' ' + df['benefits'] + ' ' + df['employment_type'] + \
    ' ' + df['required_education'] + ' ' + df['industry'] + ' ' + df['function'] 

    del df['title']
    del df['location']
    del df['department']
    del df['company_profile']
    del df['description']
    del df['requirements']
    del df['benefits']
    del df['employment_type']
    del df['required_experience']
    del df['required_education']
    del df['industry']
    del df['function']  
    
    del df['salary_range']
    del df['job_id']
    del df['telecommuting']
    del df['has_company_logo']
    del df['has_questions']

    return df

In [ ]:
df = fj_load_df_from_url()
df = fj_txt_only(df)
print('Maximum text length', df['text'].str.len().max())

Loaded dataframe shape (17880, 18)
Not fraudulent 17014 Fraudulent 866
             job_id  telecommuting  ...  has_questions    fraudulent
count  17880.000000   17880.000000  ...   17880.000000  17880.000000
mean    8940.500000       0.042897  ...       0.491723      0.048434
std     5161.655742       0.202631  ...       0.499945      0.214688
min        1.000000       0.000000  ...       0.000000      0.000000
25%     4470.750000       0.000000  ...       0.000000      0.000000
50%     8940.500000       0.000000  ...       0.000000      0.000000
75%    13410.250000       0.000000  ...       1.000000      0.000000
max    17880.000000       1.000000  ...       1.000000      1.000000

[8 rows x 5 columns]
NAs/NANs in data =>
job_id                     0
title                      0
location                 346
department             11547
salary_range           15012
company_profile         3308
description                1
requirements            2695
benefits                7210
telec

In [ ]:
# Utilities to clean text

def remove_URL(text):
    url = re.compile(r"https?://\S+|www\.\S+")
    return url.sub(r"", text)

def remove_html(text):
    html = re.compile(r"<.*?>")
    return html.sub(r"", text)

def remove_emoji(string):
    emoji_pattern = re.compile(
        "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        "]+",
        flags=re.UNICODE,
    )
    return emoji_pattern.sub(r"", string)

def remove_punct(text):
    table = str.maketrans("", "", string.punctuation)
    return text.translate(table)

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
stop = set(stopwords.words("english"))

def remove_stopwords(text):
    text = [word.lower() for word in text.split() if word.lower() not in stop]

    return " ".join(text)

In [ ]:
# clean text
df['text'] = df['text'].map(lambda x: remove_URL(x))
df['text'] = df['text'].map(lambda x: remove_html(x))
df['text'] = df['text'].map(lambda x: remove_emoji(x))
df['text'] = df['text'].map(lambda x: remove_punct(x))
df['text'] = df["text"].map(remove_stopwords)

In [ ]:
# Count unique words
def counter_word(text):
    count = Counter()
    for i in text.values:
        for word in i.split():
            count[word] += 1
    return count

text = df['text']

counter = counter_word(text)
num_words = len(counter)
print ("Number of words: ", num_words)

# Max number of words in a sequence
max_length = 250

Number of words:  172475


In [ ]:
# train-test split
train_text, test_text, train_labels , test_labels = train_test_split(df['text'], df['fraudulent'] , test_size = 0.15)

In [ ]:
# max_num_words variable in case we want to clip the number of words
max_num_words = num_words

tokenizer = Tokenizer(num_words=max_num_words)
tokenizer.fit_on_texts(train_text)

word_index = tokenizer.word_index

train_sequences = tokenizer.texts_to_sequences(train_text)
train_padded = pad_sequences(
    train_sequences, maxlen=max_length, padding="post", truncating="post"
)

test_sequences = tokenizer.texts_to_sequences(test_text)
test_padded = pad_sequences(
    test_sequences, maxlen=max_length, padding="post", truncating="post"
)

print(f"Shape of train {train_padded.shape}")
print(f"Shape of test {test_padded.shape}")

Shape of train (15198, 250)
Shape of test (2682, 250)


In [ ]:
model = Sequential()

embed_dim = 50
hidden_size = 32
model.add(Embedding(num_words, embed_dim, input_length=max_length))
model.add(Bidirectional(LSTM(hidden_size, dropout=0.1, recurrent_dropout=0.1, return_sequences=True)))
model.add(GlobalMaxPool1D())
model.add(Dense(1, activation='sigmoid'))

optimizer = Adam(learning_rate=0.01)

model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy', metrics.FalsePositives(), metrics.FalseNegatives()])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 250, 50)           8623750   
_________________________________________________________________
bidirectional (Bidirectional (None, 250, 64)           21248     
_________________________________________________________________
global_max_pooling1d (Global (None, 64)                0         
_________________________________________________________________
dense (Dense)                (None, 1)                 65        
Total params: 8,645,063
Trainable params: 8,645,063
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(train_padded, train_labels, epochs=5)

Epoch 1/5
475/475 [==============================] - 269s 567ms/step - loss: 0.1035 - accuracy: 0.9683 - false_positives: 77.0000 - false_negatives: 405.0000
Epoch 2/5
475/475 [==============================] - 269s 567ms/step - loss: 0.0226 - accuracy: 0.9932 - false_positives: 27.0000 - false_negatives: 76.0000
Epoch 3/5
475/475 [==============================] - 268s 564ms/step - loss: 0.0033 - accuracy: 0.9992 - false_positives: 3.0000 - false_negatives: 9.0000
Epoch 4/5
475/475 [==============================] - 268s 564ms/step - loss: 7.2416e-04 - accuracy: 0.9998 - false_positives: 2.0000 - false_negatives: 1.0000
Epoch 5/5
475/475 [==============================] - 270s 568ms/step - loss: 9.2114e-05 - accuracy: 1.0000 - false_positives: 0.0000e+00 - false_negatives: 0.0000e+00


In [ ]:
pred = model.predict(test_padded)
pred = np.around(pred, decimals = 0)

acc = accuracy_score(pred, test_labels)
f1 = f1_score(pred, test_labels)
print('Accuracy score: {:.4f}'.format(acc), 'F1 score: {:.4f}'.format(f1))

Accuracy score: 0.9877 F1 score: 0.8716


In [ ]:
pred_soft = model.predict(test_padded)

In [ ]:
# pred = np.around(pred_soft, decimals = 0)
pred = np.where(pred_soft > 0.50, 1, 0)

acc = accuracy_score(pred, test_labels)
f1 = f1_score(pred, test_labels)

cm = confusion_matrix(test_labels, pred)
tn = cm[0][0]
fn = cm[1][0]
tp = cm[1][1]
fp = cm[0][1]

print('Accuracy score: {:.4f}'.format(acc), 'F1 score: {:.4f}'.format(f1))
print('False Positives: {:.0f}'.format(fp), 'False Negatives: {:.0f}'.format(fn))
print('Confusion matrix:\n', cm)

Accuracy score: 0.9877 F1 score: 0.8716
False Positives: 1 False Negatives: 32
Confusion matrix:
 [[2537    1]
 [  32  112]]


In [ ]:
"""
# Uncomment to save results on drive to a csv file
df_results1 = pd.DataFrame(data=test_labels)
df_results1.reset_index(drop=True, inplace=True)
df_results2 = pd.DataFrame(data=pred_soft, columns=['lstm'])
df_results = pd.concat([df_results1, df_results2], axis=1)

from google.colab import drive
drive.mount('/content/drive')
results_file='/content/drive/My Drive/Results/lstm.csv'

df_results.to_csv(results_file)
"""

Mounted at /content/drive
